In [6]:
!pip -q uninstall -y datasets
!pip -q install "datasets<4.0.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.4 MB/s eta 0:00:00


In [4]:
!pip uninstall -y datasets -q


In [1]:
!pip install -q \
  datasets==2.19.2 \
  transformers sentencepiece accelerate \
  sacrebleu nltk bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.3.1 which is incompatible.


In [2]:
from datasets import load_dataset

ds = load_dataset(
    "Helsinki-NLP/tatoeba_mt",
    "eng-spa",
    trust_remote_code=True,
    verification_mode="no_checks",     # split size mismatch fix
    cache_dir="/content/hf_cache"
)

print("Splits:", list(ds.keys()))
print(ds)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/16582 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/197531 [00:00<?, ? examples/s]

Splits: ['test', 'validation']
DatasetDict({
    test: Dataset({
        features: ['sourceLang', 'targetlang', 'sourceString', 'targetString'],
        num_rows: 16582
    })
    validation: Dataset({
        features: ['sourceLang', 'targetlang', 'sourceString', 'targetString'],
        num_rows: 197298
    })
})


In [3]:
base_split = "train" if "train" in ds else "validation"
base = ds[base_split]

N = 1000
subset = base.shuffle(seed=42).select(range(N))

print("Using split:", base_split)
print("Base size:", len(base))
print("Subset size:", len(subset))
print("Example raw row:", subset[0])


Using split: validation
Base size: 197298
Subset size: 1000
Example raw row: {'sourceLang': 'eng', 'targetlang': 'spa', 'sourceString': 'He took my umbrella without bothering to ask.', 'targetString': 'Él tomó mi paraguas sin molestarse en preguntar.'}


In [4]:
processed = subset.map(
    lambda ex: {
        "src": ex["targetString"].strip(),  # Spanish
        "tgt": ex["sourceString"].strip(),  # English
    },
    remove_columns=subset.column_names
)

print(processed[0])


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'src': 'Él tomó mi paraguas sin molestarse en preguntar.', 'tgt': 'He took my umbrella without bothering to ask.'}


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/m2m100_418M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Device:", device)

# language codes
tokenizer.src_lang = "es"
forced_bos_token_id = tokenizer.get_lang_id("en")
print("forced_bos_token_id:", forced_bos_token_id)


Device: cpu
forced_bos_token_id: 128022


In [6]:
def m2m_translate_one(text, max_len=128, num_beams=4):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_len).to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_length=max_len,
            num_beams=num_beams,
            forced_bos_token_id=forced_bos_token_id
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

for i in range(5):
    src = processed[i]["src"]
    ref = processed[i]["tgt"]
    pred = m2m_translate_one(src)
    print("\nSRC:", src)
    print("PRED:", pred)
    print("REF:", ref)



SRC: Él tomó mi paraguas sin molestarse en preguntar.
PRED: He took my paraguas without bothering asking.
REF: He took my umbrella without bothering to ask.

SRC: Tom la salvó del incendio.
PRED: Tom saved her from the fire.
REF: Tom saved her from the fire.

SRC: Un virus infectó la computadora de Tomás.
PRED: A virus infected Tom's computer.
REF: A virus infected Tom's computer.

SRC: Los científicos más exitosos son los que plantean las preguntas correctas.
PRED: The most successful scientists are those who ask the right questions.
REF: The most successful scientists are those who pose the right questions.

SRC: Espero que apruebe el examen fácilmente.
PRED: I hope you will read the exam easily.
REF: I expect her to pass the examination easily.


In [7]:
from tqdm.auto import tqdm

def m2m_translate_batch(texts, batch_size=16, max_len=128, num_beams=4):
    preds = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len
        ).to(device)

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_length=max_len,
                num_beams=num_beams,
                forced_bos_token_id=forced_bos_token_id
            )

        preds.extend(tokenizer.batch_decode(out, skip_special_tokens=True))
    return preds

src_texts = [processed[i]["src"] for i in range(len(processed))]
refs = [processed[i]["tgt"] for i in range(len(processed))]

preds_m2m = m2m_translate_batch(src_texts, batch_size=16)
print("Pred count:", len(preds_m2m))
print("Example:", src_texts[0], "=>", preds_m2m[0])


  0%|          | 0/63 [00:00<?, ?it/s]

Pred count: 1000
Example: Él tomó mi paraguas sin molestarse en preguntar. => He took my paraguas without bothering asking.


In [8]:
import pandas as pd

df = pd.DataFrame({
    "src_es": src_texts,
    "ref_en": refs,
    "pred_m2m100_es_en": preds_m2m
})

df.to_csv("m2m100_es_en_preds.csv", index=False, encoding="utf-8")
df.head()


,src_es,ref_en,pred_m2m100_es_en
0,Él tomó mi paraguas sin molestarse en preguntar.,He took my umbrella without bothering to ask.,He took my paraguas without bothering asking.
1,Tom la salvó del incendio.,Tom saved her from the fire.,Tom saved her from the fire.
2,Un virus infectó la computadora de Tomás.,A virus infected Tom's computer.,A virus infected Tom's computer.
3,Los científicos más exitosos son los que plant...,The most successful scientists are those who p...,The most successful scientists are those who a...
4,Espero que apruebe el examen fácilmente.,I expect her to pass the examination easily.,I hope you will read the exam easily.


In [9]:
import sacrebleu
bleu = sacrebleu.corpus_bleu(preds_m2m, [refs]).score
print("M2M-100 BLEU:", bleu)


M2M-100 BLEU: 49.09177774784414


In [10]:
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

from nltk.translate.meteor_score import meteor_score

meteor = sum(
    meteor_score([r.split()], p.split())
    for r, p in zip(refs, preds_m2m)
) / len(refs)

print("M2M-100 METEOR (%):", meteor * 100)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


M2M-100 METEOR (%): 65.86120568628169


In [11]:
from bert_score import score

P, R, F1 = score(preds_m2m, refs, lang="en", verbose=True)
print("M2M-100 BERTScore F1:", F1.mean().item())


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/29 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 196.69 seconds, 5.08 sentences/sec
M2M-100 BERTScore F1: 0.9670924544334412


In [12]:
print(f"M2M-100 | BLEU: {bleu:.2f} | METEOR(%): {meteor*100:.2f} | BERTScore F1: {F1.mean().item():.4f}")


M2M-100 | BLEU: 49.09 | METEOR(%): 65.86 | BERTScore F1: 0.9671
